In [16]:
import random
import numpy as np
from collections import deque

In [21]:
# Define constants
ARRIVAL_RATE = 20/60  # Average arrivals per second
SERVICE_RATE = (1/3)/60  # Average service time is 3 minutes, so 1/3 services in 1 min
SIMULATION_TIME = 120*60  # Total simulation time in seconds
NUM_SERVERS = 45  # Number of servers

# Initialize lists for tracking customers
customer_data = []  # Active customers
completed_customers = []  # Completed customers
cycle_call_counts = []  # Call counts per cycle

# Helper function to generate patience time T
def generate_patience_time(case):
    if case == "uniform":
        return random.uniform(0, 6)  # Uniformly distributed [0, 6] minutes
    elif case == "hyperexponential":
        if random.random() < 0.5:  # Probability 1/2
            return np.random.exponential(1)  # Mean = 1 minute
        else:
            return np.random.exponential(5)  # Mean = 5 minutes

# Helper function to generate inter-arrival times
def generate_interarrival_time():
    return np.random.exponential(1 / ARRIVAL_RATE)  # Poisson arrivals

# Helper function to generate service times
def generate_service_time():
    return np.random.exponential(1 / SERVICE_RATE)  # Exponential service times

# Simulation function
def simulate(case, simulation_time=SIMULATION_TIME, num_servers=NUM_SERVERS):
    global customer_data, completed_customers, cycle_call_counts
    customer_data = []
    completed_customers = []
    cycle_call_counts = []

    # Initialize simulation variables
    current_time = 0
    next_arrival_time = generate_interarrival_time()
    servers = [None] * num_servers  # Track busy/free servers
    queue = deque()  # Call waiting queue
    customer_id = 0
    cycle_calls = 0

    while current_time < simulation_time:
        # Check if a server is done with its current call
        for i in range(num_servers):
            if servers[i] and servers[i]['service_end_time'] <= current_time:
                completed_customers.append(servers[i]['customer'])
                servers[i] = None

        # Handle queue (update waiting times and check for abandonment)
        for customer in queue.copy():
            customer['Time left in queue'] -= 1
            if customer['Time left in queue'] <= 0:
                customer['Time left in queue'] = 0
                customer['Exit time'] = current_time
                completed_customers.append(customer)
                queue.remove(customer)

        # Assign available servers to customers in the queue
        for i in range(num_servers):
            if not servers[i] and queue:
                customer = queue.popleft()
                service_time = generate_service_time()
                customer['Exit time'] = current_time + service_time
                servers[i] = {
                    'customer': customer,
                    'service_end_time': current_time + service_time,
                }

        # Handle arrivals
        if current_time >= next_arrival_time:
            # Start a new cycle if all servers are idle and queue is empty
            if all(s is None for s in servers) and not queue:
                if cycle_calls > 0:
                    cycle_call_counts.append(cycle_calls)
                cycle_calls = 0

            # Generate a new call
            customer_id += 1
            patience_time = generate_patience_time(case)
            arrival_time = current_time
            time_left_in_queue = 60 * patience_time

            new_customer = {
                'Customer_id': customer_id,
                'Time left in queue': time_left_in_queue,
                'Arrival time': arrival_time,
                'Exit time': None,
            }
            customer_data.append(new_customer)
            queue.append(new_customer)
            cycle_calls += 1

            # Schedule next arrival
            next_arrival_time += generate_interarrival_time()

        # Increment simulation time
        current_time += 1

    # Final cycle count
    if cycle_calls > 0:
        cycle_call_counts.append(cycle_calls)

# Run simulations for both cases
simulate("uniform")
uniform_case_results = {
    "customer_data": customer_data,
    "completed_customers": completed_customers,
    "cycle_call_counts": cycle_call_counts,
}

simulate("hyperexponential")
hyperexp_case_results = {
    "customer_data": customer_data,
    "completed_customers": completed_customers,
    "cycle_call_counts": cycle_call_counts,
}


In [22]:
uniform_case_results["customer_data"]

[{'Customer_id': 1,
  'Time left in queue': 207.95031187808206,
  'Arrival time': 1,
  'Exit time': 419.3480704756792},
 {'Customer_id': 2,
  'Time left in queue': 100.56661920519136,
  'Arrival time': 2,
  'Exit time': 93.10743614357325},
 {'Customer_id': 3,
  'Time left in queue': 256.15768697106677,
  'Arrival time': 3,
  'Exit time': 128.88052557391262},
 {'Customer_id': 4,
  'Time left in queue': 358.6056815300981,
  'Arrival time': 4,
  'Exit time': 110.98625416373619},
 {'Customer_id': 5,
  'Time left in queue': 208.81391037399518,
  'Arrival time': 8,
  'Exit time': 100.09961608104594},
 {'Customer_id': 6,
  'Time left in queue': 339.01612944579784,
  'Arrival time': 10,
  'Exit time': 108.82215210244786},
 {'Customer_id': 7,
  'Time left in queue': 353.062512747321,
  'Arrival time': 17,
  'Exit time': 193.144953877731},
 {'Customer_id': 8,
  'Time left in queue': 81.4507823838731,
  'Arrival time': 22,
  'Exit time': 247.67651341711982},
 {'Customer_id': 9,
  'Time left in qu